### 初始化全局变量，导入包

In [ ]:
NEED_TARGET_COUNT = True
TARGET_COUNT = 10300
MAX_WORKERS = 150
SIGNAL_MODEL_VOTE_NUM = 3
output_tag="vote"
NEED_RAG =False

In [ ]:
import os
import sys
from model import call_huoshan,call_openai
import pandas as pd
import json 
if "__file__" in globals():
    os.chdir(os.path.dirname(os.path.abspath(__file__)))

raw_data_path= os.path.join("raw_data")

sciKnowEval_path = os.path.join(raw_data_path, f"SciKnowEval_processed_openended_filling_final_merged.json")
if sciKnowEval_path.endswith(".jsonl"):
    with open(sciKnowEval_path, "r") as f:
        sciKnowEval_data= [json.loads(line) for line in f if line.strip()]
elif sciKnowEval_path.endswith(".json"):
    with open(sciKnowEval_path, "r") as f:
        sciKnowEval_data= json.load(f)
else:
    raise ValueError(f"Unsupported file format: {sciKnowEval_path}")

In [ ]:
# new_list = []
# for item in sciKnowEval_data:
#     subtask= item["metadata"]["details"]["subtask"]
#     if subtask == "Electromagnetism":
#         new_list.append(item)
# sciKnowEval_data = new_list

#### 采集各类问题

In [ ]:

# from collections import defaultdict
# sum_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# sample_list=[]
# ## 各个subtask采集5个数据
# for entry in sciKnowEval_data:
#     subtask = entry["metadata"]["details"]["subtask"]
#     domain = entry["metadata"]["domain"]
#     task= entry["metadata"]["details"]["task"]
#     if sum_dict[domain][task][subtask] < 30:
#         sum_dict[domain][task][subtask] += 1
#         sample_list.append(entry)
# print(f"Total number of entries after filtering: {len(sample_list)}")
# output_file = os.path.join(raw_data_path, f"data_sample.json")
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(sample_list, f, ensure_ascii=False, indent=4)
    
# sciKnowEval_data=sample_list

#### rag检索维基百科数据

In [ ]:
import time
import json # 导入 json 模块
from pyserini.search.lucene import LuceneSearcher
try:
    searcher = LuceneSearcher.from_prebuilt_index('wikipedia-dpr')
except Exception as e:
    print(f"初始化失败: {e}")
    # 增加一个清理缓存的提示，以防下载的索引损坏
    print("如果下载或解压出错，可以尝试删除缓存后重试： rm -rf ~/.pyserini/indexes/")
    exit()

def get_rag_input(query):

    hits = searcher.search(query, k=5)

    retrieved_passages = []
    for i, hit in enumerate(hits):

        raw_json_string = hit.raw
        passage_text = None  # 默认为 None

        if raw_json_string:
            try:
                # 解析 JSON 字符串
                doc_data = json.loads(raw_json_string)
                # 提取 'contents' 字段
                passage_text = doc_data.get('contents')
            except json.JSONDecodeError:
                print(f"警告：无法解析文档ID {hit.docid} 的JSON数据。")

        # print(f"\n--- [结果 {i+1}] ---")
        # print(f"文档 ID: {hit.docid}")
        # print(f"BM25 得分: {hit.score:.5f}")
        # print(f"段落内容:\n{passage_text}")
        
        # 将有效的内容添加到列表中，以便后续使用
        if passage_text:
            retrieved_passages.append(passage_text)

    # ================= 5. RAG 的下一步 (修正后的逻辑) =================
    # 使用我们安全收集到的段落构建上下文
    retrieved_context = "\n\n".join(retrieved_passages)

    return retrieved_context


#### 处理sciKnowEval数据：生成generation，调用verifier，整合成符合要求的最终dict格式

In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
import json_repair
from collections import Counter
Lock = Lock()
new_list= []

def sciKnowEval_process_row(row):
    
    # if len(row["generations"]) >=5:
    #     return
    # task_type = row["metadata"]["task_type"]
    id= row["id"]
    final_answer = row["ground_truth"]["final_answer"]

    question = row["question"]
    domain = row["metadata"]["domain"]
    task=row["metadata"]["details"]["task"]
    subtask = row["metadata"]["details"]["subtask"]
    ground_truth = row["ground_truth"]["final_answer"]
    
    
#     Related data retrieved from the wikipedia-dpr database:
# {{{get_rag_input(question)}}}
    # generations:list= row["generations"]
    # generations = row["generations"] 
    prompt=f"""
You are a pragmatic Data Quality Verifier. Your task is to evaluate a given ground_truth answer to determine if it is a reliable "gold standard" for automated checking.

[Input Data Start]
Question:
{{{question}}}

Ground Truth Answer:
{{{ground_truth}}}


[Input Data End]

Evaluation Instructions:

Your decision must be one of three options: Accept, Reject, or Indeterminate.

Criteria for "Accept":

The ground_truth provides a factually correct and unambiguous final answer to the question's main objective.
Minor omissions in derivation or explanation are acceptable if the core result is accurate.
Minor Formatting Flexibility is Allowed: The answer is still "Acceptable" even if it has minor formatting deviations from the question's strict instructions, as long as these deviations are made in good faith to represent all the necessary information clearly and accurately. 
Criteria for "Reject":

The final answer provided in the ground_truth is factually incorrect.
The ground_truth fails to provide a final answer to the question's main objective.
The ground_truth itself is vague, an opinion, or a clear "hallucination".
Criteria for "Indeterminate":

Choose this option if the ground_truth's correctness cannot be confidently verified without highly specialized, deep domain knowledge or external tools (e.g., running a simulation, performing a live database search, or solving complex scientific problems).
This applies when the answer makes a specific, non-trivial claim that seems plausible but is beyond the scope of a general AI's verification capabilities.
Output Format:

Strictly output your evaluation as a single JSON object. Do not add any other characters outside of the JSON structure.

```
{{
  "decision": "<'Accept', 'Reject', or 'Indeterminate'>",
  "justification": "<A brief, one-sentence explanation for your decision>"
}}
```
Your Output:
    """
    



    with ThreadPoolExecutor(max_workers=2) as executor:

        # doubao_future = executor.submit(call_huoshan, prompt, "doubao")
        # r1_future = executor.submit(call_huoshan, prompt, "r1")
        
        r1_future=[executor.submit(call_huoshan, prompt, "r1") for _ in range(SIGNAL_MODEL_VOTE_NUM)]
        doubao_future=[executor.submit(call_huoshan, prompt, "doubao") for _ in range(SIGNAL_MODEL_VOTE_NUM)]
        qwen_future = [executor.submit(call_huoshan, prompt, "qwen") for _ in range(SIGNAL_MODEL_VOTE_NUM)]

        # _, doubao_ans = doubao_future.result()
        # _, r1_ans = r1_future.result()
        doubao_ans = [future.result() for future in doubao_future]
        r1_ans = [future.result() for future in r1_future]
        qwen_ans = [future.result() for future in qwen_future]
        doubao_ans = [json_repair.repair_json(ans,return_objects=True) for _,ans in doubao_ans]
        r1_ans = [json_repair.repair_json(ans,return_objects=True) for _,ans in r1_ans]
        qwen_ans = [json_repair.repair_json(ans, return_objects=True) for _, ans in qwen_ans]
        model_ans = doubao_ans + r1_ans + qwen_ans
        decisions_list=[]
        for ans in model_ans:
            try:
                decision = ans["decision"]
                if decision not in ["Accept", "Reject", "Indeterminate"]:
                    continue
                decisions_list.append(decision)
            except KeyError:
                print(f"KeyError in ans: {ans}")
                continue
        counts = Counter(decisions_list)
        most_common_decision = max(counts, key=counts.get)

    # doubao_ans= json_repair.repair_json(doubao_ans,return_objects=True)
    # r1_ans= json_repair.repair_json(r1_ans,return_objects=True)
    res_dict={
        "id": id,
        "domain": domain,
        "task": task,
        "subtask": subtask,
        "question": question,
        "ground_truth": ground_truth,
        "model_ans": {
            "doubao": doubao_ans,
            "r1": r1_ans,
            "qwen": qwen_ans
        },
        "counts": counts,
        "decision": most_common_decision
    }
    new_list.append(res_dict)
    
    return res_dict
        


In [7]:
import traceback
import pdb
import tqdm
from threading import Lock
res_list = []


file_lock= Lock()
# 将结果写入JSON文件
file_tag="repair"
output_file = os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score.jsonl")
executor = ThreadPoolExecutor(max_workers=MAX_WORKERS)
try:

    # 2. 在 try 块内部提交和处理任务
    counter = 0
    futures = {executor.submit(sciKnowEval_process_row, row): index for index, row in enumerate(sciKnowEval_data)}
    for future in tqdm.tqdm(as_completed(futures), total=len(futures), desc="Processing rows"):
        index = futures[future]
        try:
            res=future.result()
            if res != -1:
                counter += 1
            #     json_str = json.dumps(res, ensure_ascii=False)
            #     with file_lock:
            #         try:
            #             f.write(json_str + "\n")
            #         except Exception as e:
            #             print(f"Error writing to file {output_file}: {e}")
            #             # pdb.set_trace()
                
            # if counter % 1000 == 0 and counter != 0:
            #     # print(f"Processed {counter} rows.")
            #     with file_lock, list_lock:
            #         try:
            #             with open(output_file, "w", encoding="utf-8") as f:
            #                 json.dump(res_list, f, ensure_ascii=False, indent=4)
            #         except Exception as e:
            #             print(f"Error writing to file {output_file}: {e}")
            #             # pdb.set_trace()     
            
            #### 当达到目标时中断循环
  
            if counter >= TARGET_COUNT and NEED_TARGET_COUNT:
                print(f"已达到目标数量 {TARGET_COUNT}，中断任务循环。")
                break
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            traceback.print_exc()

finally:
    print("正在关闭线程池，不再等待剩余的慢任务...")
    executor.shutdown(wait=False, cancel_futures=True) # 这是唯一被调用的shutdown
    # try:
    #     print(f"\n任务已中断或完成，最终获取了 {len(res_list)} 个结果。")
    #     with open(output_file, "w", encoding="utf-8") as f:
    #         json.dump(res_list, f, ensure_ascii=False, indent=4)
    #     print("文件保存成功！")
    # except Exception as e:
    #     print(f"Error writing to file {output_file}: {e}")
    #     pdb.set_trace()


Processing rows:  99%|█████████▉| 10308/10415 [4:14:50<02:38,  1.48s/it]

已达到目标数量 10300，中断任务循环。
正在关闭线程池，不再等待剩余的慢任务...


In [8]:
score_path = os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score_{output_tag}.json")
try:
    with open(score_path, "w", encoding="utf-8") as f:
        
        json.dump(new_list, f, ensure_ascii=False, indent=4)

except Exception as e:
    print(f"Error writing to file {score_path}: {e}")
    pdb.set_trace()
    


#### 统计各个subtask正确率

In [9]:
# import os
# import json
# path=os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score_{output_tag}.json")
# with open(path, "r", encoding="utf-8") as f:
#     new_list = json.load(f)

# # 统计各个类别的拒绝率
# from collections import defaultdict
# doubao_reject_count = defaultdict(lambda : defaultdict(lambda: defaultdict(int)))
# sum_count = defaultdict(lambda : defaultdict(lambda: defaultdict(int)))
# doubao_indeterminate_count = defaultdict(lambda : defaultdict(lambda: defaultdict(int)))
# r1_reject_count = defaultdict(lambda : defaultdict(lambda: defaultdict(int)))
# r1_indeterminate_count = defaultdict(lambda : defaultdict(lambda: defaultdict(int)))
# for entry in new_list:
#     domain = entry["domain"]
#     task = entry["task"]
#     subtask = entry["subtask"]
#     doubao_decision = entry["doubao_answer"]["decision"]
#     r1_decision = entry["r1_answer"]["decision"]
#     sum_count[domain][task][subtask] += 1
#     if doubao_decision == "Reject":
#         doubao_reject_count[domain][task][subtask] += 1
#     elif doubao_decision == "Indeterminate":
#         doubao_indeterminate_count[domain][task][subtask] += 1
#     if r1_decision == "Reject":
#         r1_reject_count[domain][task][subtask] += 1
#     elif r1_decision == "Indeterminate":
#         r1_indeterminate_count[domain][task][subtask] += 1


# # for domain, tasks in reject_count.items():
# #     print(f"Domain: {domain}")
# #     for task, subtasks in tasks.items():
# #         print(f"    Task: {task}")
# #         for subtask, reject in subtasks.items():
# #             indeterminate= indeterminate_count[domain][task][subtask]
# #             total = sum_count[domain][task][subtask]
# #             if total > 0:
# #                 reject_rate = reject / total
# #                 indeterminate_rate = indeterminate / total
                
# #                 print(f"        Subtask: {subtask}, Reject Rate: {reject_rate:.2%} ({reject}/{total})")
# #                 print(f"        Subtask: {subtask}, Indeterminate Rate: {indeterminate_rate:.2%} ({indeterminate}/{total})")
# #             else:
# #                 print(f"Domain: {domain}, Task: {task}, Subtask: {subtask}, Reject Rate: N/A (No data)")

# for domain, tasks in sum_count.items():
#     print(f"Domain: {domain}")
#     for task, subtasks in tasks.items():
#         print(f"    Task: {task}")
#         for subtask, total in subtasks.items():
#             doubao_reject = doubao_reject_count[domain][task][subtask]
#             doubao_indeterminate = doubao_indeterminate_count[domain][task][subtask]
#             r1_reject = r1_reject_count[domain][task][subtask]
#             r1_indeterminate = r1_indeterminate_count[domain][task][subtask]
            
#             if total > 0:
#                 doubao_reject_rate = doubao_reject / total
#                 doubao_indeterminate_rate = doubao_indeterminate / total
#                 r1_reject_rate = r1_reject / total
#                 r1_indeterminate_rate = r1_indeterminate / total
                
#                 print(f"        Subtask: {subtask}, Doubao Reject Rate: {doubao_reject_rate:.2%} ({doubao_reject}/{total}), Doubao Indeterminate Rate: {doubao_indeterminate_rate:.2%} ({doubao_indeterminate}/{total})")
#                 print(f"        Subtask: {subtask}, R1 Reject Rate: {r1_reject_rate:.2%} ({r1_reject}/{total}), R1 Indeterminate Rate: {r1_indeterminate_rate:.2%} ({r1_indeterminate}/{total})")
#             else:
#                 print(f"Domain: {domain}, Task: {task}, Subtask: {subtask}, Reject Rate: N/A (No data)")


#### 统计投票的一致性

In [10]:
import os
import json
path=os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score_{output_tag}.json")
with open(path, "r", encoding="utf-8") as f:
    new_list = json.load(f)

# 统计投票的一致性，即最终的决策数量的分布
from collections import defaultdict
decssion_num_dict=defaultdict(int)
for entry in new_list:
    decision = entry["decision"]
    decision_num= entry["counts"][decision]
    decssion_num_dict[decision_num] += 1
# 打印投票的一致性统计
print("投票的一致性统计：")
for decision_num, count in decssion_num_dict.items():
    print(f"决策数量: {decision_num}, 数量: {count}") 
    
    

投票的一致性统计：
决策数量: 6, 数量: 893
决策数量: 7, 数量: 939
决策数量: 8, 数量: 1474
决策数量: 5, 数量: 739
决策数量: 9, 数量: 5936
决策数量: 4, 数量: 281
决策数量: 3, 数量: 38
